In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the folder path containing your folders of images
folder_path = "R:\SIEMENS_Intern\chlorine"

# List all folders in the specified directory
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

# Define the labels
labels = [0.1, 0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0]

# Create a dictionary to store the mapping of folder names to labels
label_mapping = {folder: label for folder, label in zip(folders, labels)}

# Print the mapping to verify
print("Folder Label Mapping:")
print(label_mapping)



Folder Label Mapping:
{'Sample_0.1': 0.1, 'Sample_0.2': 0.2, 'Sample_0.5': 0.5, 'Sample_1.0': 1.0, 'Sample_1.5': 1.5, 'Sample_2.0': 2.0, 'Sample_2.5': 2.5, 'Sample_3.0': 3.0, 'Sample_4.0': 4.0, 'Sample_5.0': 5.0}


In [2]:
# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

# Define train and validation data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Splitting data into train and validation
train_generator = train_datagen.flow_from_directory(
    folder_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Set as training data

validation_generator = train_datagen.flow_from_directory(
    folder_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Set as validation data

Found 1591 images belonging to 10 classes.
Found 392 images belonging to 10 classes.


In [3]:

# Define and compile the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(labels), activation='softmax')  # Number of units based on the number of categories
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the trained model
model.save("chlorine_concentration_model.h5")


Epoch 1/10
49/49 [==============================] - 170s 3s/step - loss: 2.5681 - accuracy: 0.1206 - val_loss: 2.2375 - val_accuracy: 0.1146
Epoch 2/10
49/49 [==============================] - 173s 4s/step - loss: 1.7481 - accuracy: 0.3079 - val_loss: 1.2796 - val_accuracy: 0.4531
Epoch 3/10
49/49 [==============================] - 227s 5s/step - loss: 1.0454 - accuracy: 0.5850 - val_loss: 1.0865 - val_accuracy: 0.5156
Epoch 4/10
49/49 [==============================] - 294s 6s/step - loss: 0.7619 - accuracy: 0.7011 - val_loss: 0.7525 - val_accuracy: 0.7448
Epoch 5/10
49/49 [==============================] - 293s 6s/step - loss: 0.5406 - accuracy: 0.7986 - val_loss: 0.9310 - val_accuracy: 0.5964
Epoch 6/10
49/49 [==============================] - 300s 6s/step - loss: 0.3808 - accuracy: 0.8589 - val_loss: 0.8157 - val_accuracy: 0.7214
Epoch 7/10
49/49 [==============================] - 291s 6s/step - loss: 0.3029 - accuracy: 0.8954 - val_loss: 0.8408 - val_accuracy: 0.7318
Epoch 8/10
49

In [5]:
# Classify new images
new_image_path = r'R:\SIEMENS_Intern\chlorine\Sample_2.5\aug_0_200.jpeg'
img = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(img_height, img_width))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

predictions = model.predict(img_array)
predicted_class_index = tf.argmax(predictions, axis=1)
predicted_class = labels[predicted_class_index[0]]
print("Predicted concentration:", predicted_class)


1/1 [==============================] - 1s 700ms/step
Predicted concentration: 2.5
